In [131]:
using Gridap
using GridapMakie, CairoMakie, FileIO
using Gridap.FESpaces
using Gridap.ReferenceFEs
using Gridap.Arrays
using Gridap.Algebra
using Gridap.Geometry
using Gridap.Fields
using Gridap.CellData
using FillArrays
using Test
using InteractiveUtils

In [132]:
include("Julia_functions/solution_animation.jl")
include("Julia_functions/indicator_chi.jl")

χ (generic function with 1 method)

In [154]:
domain = (0,1,0,1)
partition = (100,100)
model = CartesianDiscreteModel(domain,partition) #|> simplexify
order = 1
reffe = ReferenceFE(lagrangian,Float64,order)
Testspace = TestFESpace(model,reffe,conformity=:H1) ###### conformity correct?
Trialspace = TransientTrialFESpace(Testspace)                                # maybe add a function for/if Dirichlet conditions

Uspace = FESpace(model, reffe, conformity=:H1)

degree = 2*order                                                    # degree of the method used for approximating integrals over Ω
Ω = Triangulation(model)
dΩ = Measure(Ω,degree)      										# make the measure dΩ
Γ = BoundaryTriangulation(model)                                    # triangulate the boundary ∂Ω
dΓ = Measure(Γ,degree)   


q_pos(x) = χ(x[1], 0.9, 0.85) * χ(x[2], 0, 0.20)

ρ=1.0 #1.225
c = 1.0# c=1020.0
k = 1.0#1.0# k=15.0
h = 1.0# h=0.7
Tout(x,t)= 0.0*x[1]
Q(x,t) = 0.0#x[1] + x[2] #0.0#10.0*q_pos(x)
Tini(x) = 20.0 #1.0 * x[1] + 10.0
t0=0.0
tF=10.0
TIni=interpolate_everywhere(Tini, Uspace(t0))
Tfin=interpolate_everywhere(20.0, Uspace(tF))
Δt = 0.05

function draw(ysol)
	fig, _ , plt = CairoMakie.plot(Ω, ysol, colormap=:plasma)               # plot of last state (numerical solution)
	CairoMakie.wireframe!(Ω, color=:black, linewidth=1)                        # add triangulation
	CairoMakie.Colorbar(fig[1,2], plt)                                         # add color bar
	display(fig)															  # display the plot
end
# draw(TIni)


draw (generic function with 1 method)

In [ ]:
ls = LUSolver()
θ = 0.5
solver = ThetaMethod(ls, Δt, θ)

Q(t)=x->Q(x,t)
Tout(t)=x->Tout(x,t)

a_SE_tconst(t, dtT, ϕ) = ∫(c*dtT*ϕ*ρ)dΩ
# a_SE_tnonconst(t, T, ϕ) = (1)*∫(k * ∇(T) ⋅ ∇(ϕ) - tr(h*T*ϕ))dΩ
a_SE_tnonconst(t, T, ϕ) = ∫(k * ∇(T)⊙∇(ϕ))dΩ + ∫(h*T*ϕ)dΓ


l_SE(t, ϕ) = ∫(Q(t) * ϕ)dΩ + ∫(Tout(t) * ϕ * h)dΓ

op_SE = TransientLinearFEOperator((a_SE_tconst, a_SE_tnonconst), l_SE, Trialspace, Testspace, constant_forms=(true, true))

T = solve(solver, op_SE, t0, tF, TIni)

sol = []
count = 0
for (tn, uhn) in T
    if count%5 == 0
        push!(sol,uhn)
    end
    count += 1
    
end


In [162]:
if !isdir("tmp")
    mkdir("tmp")
  end
  
  createpvd("results") do pvd
    pvd[0] = createvtk(Ω, "tmp/results_0" * ".vtu", cellfields=["u" => TIni])
    for (tn, uhn) in T
      pvd[tn] = createvtk(Ω, "tmp/results_$tn" * ".vtu", cellfields=["u" => uhn])
    end
  end

202-element Vector{String}:
 "results.pvd"
 "tmp/results_0.vtu"
 "tmp/results_0.05.vtu"
 "tmp/results_0.1.vtu"
 "tmp/results_0.15000000000000002.vtu"
 "tmp/results_0.2.vtu"
 "tmp/results_0.25.vtu"
 "tmp/results_0.3.vtu"
 "tmp/results_0.35.vtu"
 "tmp/results_0.39999999999999997.vtu"
 ⋮
 "tmp/results_9.600000000000001.vtu"
 "tmp/results_9.650000000000002.vtu"
 "tmp/results_9.700000000000003.vtu"
 "tmp/results_9.750000000000004.vtu"
 "tmp/results_9.800000000000004.vtu"
 "tmp/results_9.850000000000005.vtu"
 "tmp/results_9.900000000000006.vtu"
 "tmp/results_9.950000000000006.vtu"
 "tmp/results_10.000000000000007.vtu"

In [163]:
fig, ax, hm = plot(sol[1])
Colorbar(fig[1,2], hm)
display(fig)

CairoMakie.Screen{IMAGE}


In [164]:
#solution_animation(sol,30)